In [1]:
from utils.read_file import read_target_insurance_pdf, read_json

In [2]:
QUESTION_PATH = "競賽資料集/dataset/preliminary/questions_example.json"

INSURANCE_PATH = "競賽資料集/reference/insurance"
FAQ_PATH = "競賽資料集/reference/faq/pid_map_content.json"
ANS_PATH = "競賽資料集/dataset/preliminary/ground_truths_example.json"

In [3]:
question = read_json(QUESTION_PATH)["questions"]

In [4]:
# from huggingface_hub import snapshot_download

# # 下載模型到指定路徑
# local_model_path = "./models/bge-m3"
# snapshot_download(repo_id="BAAI/bge-m3", revision="main", local_dir=local_model_path)

C:\Users\weiso131\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files:  60%|██████    | 18/30 [00:20<00:00, 18.21it/s]Error while downloading from https://cdn-lfs-us-1.hf.co/repos/23/2c/232ca60237b0bb19bb6c28c5a6c8af79f2e423333a9626aad445543b80fbf31e/1eebfb28493f67bba03ce0ef64bfdc7fc5a3bd9d7493f818bb1d78cd798416b4?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.onnx_data%3B+filename%3D%22model.onnx_data%22%3B&Expires=1730383139&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMDM4MzEzOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzIzLzJjLzIzMmNhNjAyMzdiMGJiMTliYjZjMjhjNWE2YzhhZjc5ZjJlNDIzMzMzYTk2MjZhYWQ0NDU1NDNiODBmYmYzMWUvMWVlYmZiMjg0OTNmNjdiYmEwM2NlMGVmNjRiZmRjN2ZjNWEzYmQ5ZDc0OTNmODE4YmIxZDc

'C:\\Users\\weiso131\\Desktop\\AI cup 玉山銀行\\models\\bge-m3'

In [5]:
from sentence_transformers import SentenceTransformer

# 加載模型
embbeded_model = SentenceTransformer("./models/bge-m3", device='cuda')

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_ans(embedded_model, question_data: dict, documents: list, k=1):
    document_ebeddings = embedded_model.encode(documents)

    # 使用者查詢
    user_query = question_data["query"]

    # 查詢文本轉換成嵌入向量
    query_embedding = embbeded_model.encode([user_query])

    # 計算相似度
    similarities = cosine_similarity(query_embedding, document_ebeddings)
    k_highest = np.argsort(similarities[0])[-k:][::-1]
    return k_highest

In [7]:

def generate_faq_retrieve(faq: dict, id: int) -> str:

    text = ""

    for faq_data in faq[str(id)]:

        question = faq_data["question"]
        answers = faq_data["answers"]
        text += f"問題:{question}, 回答:{answers}\n"
    return text

def read_target_faq(faq: dict, indexs: list) -> list:

    all_texts = []

    for i in indexs:
        all_texts.append(generate_faq_retrieve(faq, i))
    
    return all_texts


def validate_faq(embbeded_model, question):
    faq = read_json(FAQ_PATH)
    ans = read_json(ANS_PATH)
    correct = 0
    for i in range(100, 150):
        document = read_target_faq(faq, question[i]["source"])
        predict = question[i]["source"][get_ans(embbeded_model, question[i], document)[0]]
        if (predict == ans["ground_truths"][i]["retrieve"]):
            correct += 1
        else:
            print(f"qid: {i + 1}, predict: {predict}, ans: {ans["ground_truths"][i]["retrieve"]}")
    return correct / 50


### 驗證faq資料集的正確率

In [8]:
validate_faq(embbeded_model, question)

C:\Users\weiso131\AppData\Roaming\Python\Python312\site-packages\transformers\models\xlm_roberta\modeling_xlm_roberta.py:371: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


qid: 109, predict: 234, ans: 283
qid: 135, predict: 399, ans: 28


0.96

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # 每段的目標長度
    chunk_overlap=400  # 分段間的重疊字符數
)

def validate_insurance(embbeded_model, question):
    ans = read_json(ANS_PATH)
    correct = 0
    for i in range(0, 50):
        src = question[i]["source"]
        document, order_list = read_target_insurance_pdf(INSURANCE_PATH, src)
        tokens = []
        chunk_real_index = []
        for j in range(len(document)):
            text = document[j]
            chunks = text_splitter.split_text(text)
            tokens.extend(chunks)
            chunk_real_index.extend([j + k * 0 for k in range(len(chunks))])

                
        chunk_real_index=np.array(chunk_real_index)

        token_index = get_ans(embbeded_model, question[i], tokens, 5)
        real_index = chunk_real_index[token_index]
        predict = order_list[real_index[0]]


        if (predict == ans["ground_truths"][i]["retrieve"]):
            correct += 1
        else:
            print(f"qid: {i + 1}, predict: {predict}, ans: {ans["ground_truths"][i]["retrieve"]}")
            for index in token_index:
                print(order_list[chunk_real_index[index]])
            #print(tokens[token_index[0]])
    return correct / 50

### 驗證insurance資料集的正確率

In [10]:
validate_insurance(embbeded_model, question)

qid: 2, predict: 258, ans: 428
258
258
565
606
475
qid: 3, predict: 476, ans: 83
476
83
83
83
179
qid: 4, predict: 179, ans: 186
179
186
627
627
627
qid: 6, predict: 242, ans: 116
242
275
116
116
275
qid: 15, predict: 526, ans: 536
526
476
337
182
182
qid: 23, predict: 228, ans: 224
228
428
228
99
428
qid: 28, predict: 78, ans: 298
78
298
298
258
506
qid: 29, predict: 256, ans: 578
256
524
102
431
524
qid: 35, predict: 224, ans: 319
224
502
319
224
283
qid: 42, predict: 445, ans: 325
445
325
188
445
568
qid: 49, predict: 357, ans: 482
357
482
580
51
482
qid: 50, predict: 555, ans: 78
555
341
555
555
528


0.76